In [ ]:
# !apt-get update 
# !pip3 install pybullet
# !apt-get install -y ffmpeg
# https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#

In [ ]:
import pybullet as p
import time
import numpy as np
import os
import sys
from collections import namedtuple

import _utils_v2 as utils
from _utils_v2 import geom_shape, xyz_v2, s, CFG_Loader

In [ ]:
ROBOT = namedtuple('Body',['CSI', 'Masses', 'Positions', 'indices', 'jntTypes', 'axis', 'angles'])

class name(object):
    def __init__(self, 
                 path2cfgs, 
                 VARIABLE        = 0.04,
                 fig_joint_coeff = 0.25,
                 base_body_Mass  = 20000,
                 base_position   = utils.coordinates(0,0,0)):
        
        super(name, self).__init__()
        self.cfgs   = CFG_Loader(path2cfgs).get_result()
        self.m2j    = ['m','j']
        self.j2m    = ['j','m']
        self.m2m    = ['m','m']
        self.d2m    = ['d','m']
        self.m2d    = ['m','d']
        self.STATIC = [0,0,0]
        
        self.j = fig_joint_coeff
        self.VARIABLE  = VARIABLE
        self.body_Mass = base_body_Mass
        self.base_position  = base_position
        
    @staticmethod
    def dict2list(from_dict):
        return list(from_dict.values())
    
    def get_data(self, name):
        if name.startswith('R_'):
            name = name.replace('R_','')
            data = self.cfgs[name].RIGHT
            try:
                additional_name = data.motor_ops.stick_to
                if additional_name.startswith('R_'):
                    additional_name = additional_name.replace('R_','')
                    addit_data = self.cfgs[additional_name].RIGHT
                else:
                    addit_data = self.cfgs[additional_name]
            except:
                addit_data = None
        
        elif name.startswith('L_'):
            name = name.replace('L_','')
            data = self.cfgs[name].LEFT
            try:
                additional_name = data.motor_ops.stick_to
                if additional_name.startswith('L_'):
                    additional_name = additional_name.replace('L_','')
                    addit_data = self.cfgs[additional_name].LEFT
                else:
                    addit_data = self.cfgs[additional_name]
            except:
                addit_data = None
        else:
            data = self.cfgs[name]
            try:
                additional_name = data.motor_ops.stick_to
                addit_data = self.cfgs[additional_name]
            except:
                addit_data = None
        return data, addit_data

    def structure(self, name, index_J, index_M):
        """
        CSI : CollisionShapeIndexes
        """
        basic_data, \
        addit_data = self.get_data(name)
        motor_ops  = basic_data.motor_ops
        
        # print('basic_data =', basic_data)
        # print('addit_data =', addit_data)
        J = ROBOT(
            CSI       = utils.geom_shape(p.GEOM_SPHERE, radius = self.VARIABLE, coeff_R = self.j), 
            Masses    = 1e-6,  
            Positions = xyz_v2(self.m2j,addit_data, basic_data),
            indices   = index_J,
            jntTypes  = p.JOINT_REVOLUTE, 
            axis      = self.dict2list(motor_ops.axes), 
            angles    = [motor_ops.min_angle, motor_ops.max_angle]
        )
        M = ROBOT(
            CSI       = utils.geom_shape(p.GEOM_SPHERE, radius = self.VARIABLE, coeff_R = 1), 
            Masses    = basic_data.mass,  
            Positions = xyz_v2(self.j2m, basic_data),
            indices   = index_M, 
            jntTypes  = p.JOINT_PRISMATIC, 
            axis      = self.STATIC, 
            angles    = None
        )
        return J,M
            
    # =====================================================================================================
    def build_PELVIS(self):   
        self.M_Pelvis = ROBOT(
            CSI       = utils.geom_shape(p.GEOM_SPHERE, radius = self.VARIABLE), 
            Masses    = self.cfgs.Pelvis.mass, 
            Positions = xyz_v2(self.d2m,self.base_position,self.cfgs.Pelvis),
            indices   = 0, 
            jntTypes  = p.JOINT_PRISMATIC, 
            axis      = self.STATIC,
            angles    = None)
    
    def init_torso_s(self):
        self.J_TorsoS, self.M_TorsoS = self.structure('TorsoS',1,2)
    def init_torso_f(self):
        self.J_TorsoF, self.M_TorsoF = self.structure('TorsoF',3,4)
    def init_torso_r(self):
        self.J_TorsoR, self.M_TorsoR = self.structure('TorsoR',5,6)    
    def build_TORSO(self):
        self.init_torso_s()
        self.init_torso_f()
        self.init_torso_r()
        
    def BUILD_TORSO(self):
        self.build_PELVIS()
        self.build_TORSO()
    # =====================================================================================================
    def build_NECK(self):# Neck 
        self.J_Neck, self.M_Neck = self.structure('Neck',7,8)
    
    def init_head_r(self):
        self.J_HeadR, self.M_HeadR = self.structure('HeadR',9,10)
    def init_head_f(self):
        self.J_HeadF, self.M_HeadF = self.structure('HeadF',11,12)
    def build_HEAD(self):# Head
        self.init_head_r()
        self.init_head_f()
    
    def BUILD_NECK_AND_HEAD(self):
        self.build_NECK()
        self.build_HEAD()
    # =====================================================================================================
    def init_r_shoulder_f(self):
        self.R_J_ShldrF, self.R_M_ShldrF = self.structure('R_ShoulderF',7,14)
    def init_r_shoulder_s(self):
        self.R_J_ShldrS, self.R_M_ShldrS = self.structure('R_ShoulderS',15,16)
    def init_r_elbow_r(self):
        self.R_J_ElbowR, self.R_M_ElbowR = self.structure('R_ElbowR',17,18)
    def init_r_elbow(self):
        self.R_J_Elbow, self.R_M_Elbow   = self.structure('R_Elbow',19,20)
    
    def init_right_ARM(self): # Right Arm
        self.init_r_shoulder_f()
        self.init_r_shoulder_s()
        self.init_r_elbow_r()
        self.init_r_elbow()
        
    def init_l_shoulder_f(self):
        self.L_J_ShldrF, self.L_M_ShldrF = self.structure('L_ShoulderF',7,22)
    def init_l_shoulder_s(self):
        self.L_J_ShldrS, self.L_M_ShldrS = self.structure('L_ShoulderS',23,24)
    def init_l_elbow_r(self):
        self.L_J_ElbowR, self.L_M_ElbowR = self.structure('L_ElbowR',25,26)
    def init_l_elbow(self):
        self.L_J_Elbow, self.L_M_Elbow   = self.structure('L_Elbow',27,28)

    def init_left_ARM(self): # Left Arm
        self.init_l_shoulder_f()
        self.init_l_shoulder_s()
        self.init_l_elbow_r()
        self.init_l_elbow()
    
    def BUILD_ARMS(self):
        self.init_right_ARM()
        self.init_left_ARM()
    # =====================================================================================================
    def init_r_hip_f(self):
        self.R_J_HipF, self.R_M_HipF = self.structure('R_HipF', 1,30)
    def init_r_hip_s(self):
        self.R_J_HipS, self.R_M_HipS = self.structure('R_HipS',31,32)
    def init_r_hip_r(self):
        self.R_J_HipR, self.R_M_HipR = self.structure('R_HipR',33,34)
    def init_r_knee(self):
        self.R_J_Knee, self.R_M_Knee = self.structure('R_Knee',35,36)
    def init_r_ankle_f(self):
        self.R_J_AnkleF, self.R_M_AnkleF = self.structure('R_AnkleF',37,38)
    def init_r_ankle_s(self):
        self.R_J_AnkleS, self.R_M_AnkleS = self.structure('R_AnkleS',39,40)    
    
    def init_l_hip_f(self):
        self.L_J_HipF, self.L_M_HipF = self.structure('L_HipF', 1,43)
    def init_l_hip_s(self):
        self.L_J_HipS, self.L_M_HipS = self.structure('L_HipS',44,45)
    def init_l_hip_r(self):
        self.L_J_HipR, self.L_M_HipR = self.structure('L_HipR',46,47)
    def init_l_knee(self):
        self.L_J_Knee, self.L_M_Knee = self.structure('L_Knee',48,49)
    def init_l_ankle_f(self):
        self.L_J_AnkleF, self.L_M_AnkleF = self.structure('L_AnkleF',50,51)
    def init_l_ankle_s(self):
        self.L_J_AnkleS, self.L_M_AnkleS = self.structure('L_AnkleS',52,53)
    
    def init_r_foot(self):
        self.R_M_Foot = ROBOT(
            CSI       = utils.geom_shape(p.GEOM_BOX, height = self.VARIABLE, coeff_P = [1,3,0.25]), 
            Masses    = self.cfgs.Foot.RIGHT.mass, 
            Positions = xyz_v2(self.m2m,self.cfgs.AnkleS.RIGHT, self.cfgs.Foot.RIGHT),
            indices   = 41, 
            jntTypes  = p.JOINT_PRISMATIC, 
            axis      = self.STATIC,
            angles    = None)
    def init_l_foot(self):
        self.L_M_Foot = ROBOT(
            CSI       = utils.geom_shape(p.GEOM_BOX, height = self.VARIABLE, coeff_P = [1,3,0.25]), 
            Masses    = self.cfgs.Foot.LEFT.mass, 
            Positions = xyz_v2(self.m2m,self.cfgs.AnkleS.LEFT, self.cfgs.Foot.LEFT),
            indices   = 54, 
            jntTypes  = p.JOINT_PRISMATIC, 
            axis      = self.STATIC,
            angles    = None)

    def init_right_LEG(self): # Left Leg
        self.init_r_hip_f()
        self.init_r_hip_s()
        self.init_r_hip_r()
        self.init_r_knee()
        self.init_r_ankle_f()
        self.init_r_ankle_s()
        self.init_r_foot()

    def init_left_LEG(self): # Left Leg
        self.init_l_hip_f()
        self.init_l_hip_s()
        self.init_l_hip_r()
        self.init_l_knee()
        self.init_l_ankle_f()
        self.init_l_ankle_s()
        self.init_l_foot()

    def BUILD_LEGS(self):
        self.init_right_LEG()
        self.init_left_LEG()
    
    # =====================================================================================================
    def BUILD_BODY(self):
        self.BUILD_TORSO()
        self.BUILD_NECK_AND_HEAD()
        self.BUILD_ARMS()
        self.BUILD_LEGS()
        self.cfgs = [ 
            self.M_Pelvis, 
            self.J_TorsoS,   self.M_TorsoS,   
            self.J_TorsoF,   self.M_TorsoF,   
            self.J_TorsoR,   self.M_TorsoR,
            self.J_Neck,     self.M_Neck,     
            self.J_HeadR,    self.M_HeadR,    
            self.J_HeadF,    self.M_HeadF,
            self.R_J_ShldrF, self.R_M_ShldrF, 
            self.R_J_ShldrS, self.R_M_ShldrS, 
            self.R_J_ElbowR, self.R_M_ElbowR, 
            self.R_J_Elbow,  self.R_M_Elbow,
            self.L_J_ShldrF, self.L_M_ShldrF, 
            self.L_J_ShldrS, self.L_M_ShldrS, 
            self.L_J_ElbowR, self.L_M_ElbowR, 
            self.L_J_Elbow,  self.L_M_Elbow,
            self.R_J_HipF,   self.R_M_HipF,   
            self.R_J_HipS,   self.R_M_HipS,   
            self.R_J_HipR,   self.R_M_HipR,   
            self.R_J_Knee,   self.R_M_Knee, 
            self.R_J_AnkleF, self.R_M_AnkleF, 
            self.R_J_AnkleS, self.R_M_AnkleS, 
            self.R_M_Foot,   
            self.L_J_HipF,   self.L_M_HipF,   
            self.L_J_HipS,   self.L_M_HipS,   
            self.L_J_HipR,   self.L_M_HipR,   
            self.L_J_Knee,   self.L_M_Knee,
            self.L_J_AnkleF, self.L_M_AnkleF, 
            self.L_J_AnkleS, self.L_M_AnkleS, 
            self.L_M_Foot
        ]
    
    def create_body(self):
        linkCollisionShapeIndices = [p[0] for p in self.cfgs]
        link_Masses               = [p[1] for p in self.cfgs]
        linkPositions             = [p[2] for p in self.cfgs]
        indices                   = [p[3] for p in self.cfgs]
        jointTypes                = [p[4] for p in self.cfgs]
        axis                      = [p[5] for p in self.cfgs]
        angles                    = [p[6] for p in self.cfgs] 
        
        Torso     = [1,3,5] 
        NandH     = [7,9,11] 
        Right_arm = [13,15,17,19]
        Left_arm  = [21,23,25,27]
        Right_leg = [29,31,33,35,37,39]
        Left_leg  = [42,44,46,48,50,52]

        # BLOCK Init
        top = Torso + NandH + Right_arm + Left_arm
        bot = Right_leg + Left_leg

        # default params 
        motorIndexes = top + bot
        linkValues   = [0] * len(motorIndexes)

        visualShapeId   = -1
        nlnk            = len(link_Masses)
        LVSIndices      = [-1] * nlnk 
        LOrientations   = [[0,0,0,1]] * nlnk
        LIFPositions    = [[0,0,0]]   * nlnk
        LIFOrientations = [[0,0,0,1]] * nlnk
        #Drop the body in the scene at the following body coordinates
        baseOrientation = [0,0,0,1]
        robot_positions = [[0,0,1]] # b_p.Z 

        for basePosition in robot_positions:
            robot = p.createMultiBody(
                baseMass                      = self.body_Mass,
                baseCollisionShapeIndex       = utils.geom_shape(p.GEOM_BOX, 
                                                              height  = 0.04, 
                                                              coeff_P = [10,10,0.1]),
                baseVisualShapeIndex          = visualShapeId,
                basePosition                  = basePosition,
                baseOrientation               = baseOrientation,

                linkVisualShapeIndices        = LVSIndices,
                linkOrientations              = LOrientations,
                linkInertialFramePositions    = LIFPositions,
                linkInertialFrameOrientations = LIFOrientations,

                linkMasses                    = link_Masses,
                linkCollisionShapeIndices     = linkCollisionShapeIndices,
                linkPositions                 = linkPositions,
                linkParentIndices             = indices,
                linkJointTypes                = jointTypes,
                linkJointAxis                 = axis
            )
            utils.resetCoordinates(robot, motorIndexes, linkValues)

        # Add earth like gravity
        p.setGravity(0,0,-9.81)
        p.setRealTimeSimulation(1)

        cameraDistance = 5.0
        cameraYaw      = 180
        cameraPitch    = -27.60

        utils.resetCamera(
            cameraDistance = cameraDistance,
            cameraYaw      = cameraYaw,
            cameraPitch    = cameraPitch,
        )
        
    def start(self):
        # init window
        utils.init_window()
        # create shapes
        self.BUILD_BODY()
        self.create_body()
    
    def stop(self):
        p.disconnect()
    
path = './cfgs'
body = name(path)

In [ ]:
body.start()

In [ ]:
body.stop()

In [ ]:
p.disconnect()

In [ ]:
angle_HeadF       = 0; angle_HeadR       = 0; angle_neck   = 0;
angle_TorsoS      = 0; angle_TorsoF      = 0; angle_TorsoR = 0;
angle_R_ShoulderF = 0; angle_L_ShoulderF = 0
angle_R_ShoulderS = 0; angle_L_ShoulderS = 0
angle_R_ElbowR    = 0; angle_L_ElbowR    = 0
angle_R_Elbow     = 0; angle_L_Elbow     = 0
angle_R_HipF      = 0; angle_L_HipF      = 0
angle_R_HipS      = 0; angle_L_HipS      = 0
angle_R_HipR      = 0; angle_L_HipR      = 0
angle_R_Knee      = 0; angle_L_Knee      = 0
angle_R_AnkleF    = 0; angle_L_AnkleF    = 0
angle_R_AnkleS    = 0; angle_L_AnkleS    = 0

In [ ]:
TorsoS      = [ -15,  15]   
TorsoF      = [ -12,  40] 
TorsoR      = [ -95,  95]
Neck        = [ -10,  40]   
HeadR       = [ -80,  80] 
HeadF       = [ -60,  20]
ShoulderF   = [-150,  90] 
L_ShoulderS = [   0,-150] 
R_ShoulderS = [ 150,   0]
ElbwR       = [-110, 110] 
Elbw        = [ 120,   5]
__HipF      = [ -90,  35]   
R_HipS      = [ -90,   2] 
L_HipS      = [ -12,  90] 
__HipR      = [ -90,  90]
__Knee      = [   0, 110]   
__AnkleF    = [ -40,  25] 
__AnkleS    = [ -30,  30]

In [ ]:
from MAIN_LOOP.printer import printer
from MAIN_LOOP.options import options
from MAIN_LOOP.body_mv import move_head, move_torso, move_arms, move_legs

# angleFHead, angleRHead, angle_neck
main_flag = True
while main_flag:
    msg = 'Main: Options(O), BodyMove(M), Info(?), Quit(Q)'
    printer(msg)
    keys = p.getKeyboardEvents()
    if keys.get(111): # o - options
        result = options(cameraDistance, cameraYaw, cameraPitch)
        cameraDistance, cameraYaw, cameraPitch = result
    if keys.get(109): # m - move body paths
        move_flag = True
        while move_flag:
            printer('Move: Head(H), Torso(T), Arms(A), Legs(L), Info(?). Quit(1)')
            move_loop = p.getKeyboardEvents()
            if move_loop.get(104):# H - Head
                angles      = [angle_neck, angle_HeadR, angle_HeadF]
                minmaxNandH = [Neck, HeadR, HeadF]
                angles      = move_head(robot, NandH, angles, minmaxNandH)
            if move_loop.get(116):# T - Torso
                angles      = [angle_TorsoS, angle_TorsoF, angle_TorsoR]
                minmaxTorso = [TorsoS,TorsoF,TorsoR]
                angles      = move_torso(robot, Torso, angles, minmaxTorso)
            if move_loop.get(97): # A - Arms
                angles      = [
                    angle_R_ShoulderF, angle_L_ShoulderF,
                    angle_R_ShoulderS, angle_L_ShoulderS,
                    angle_R_ElbowR,    angle_L_ElbowR,
                    angle_R_Elbow,     angle_L_Elbow,]
                minmaxArms  = [ShoulderF, [R_ShoulderS, L_ShoulderS], ElbwR, Elbw]
                Arms        = ShldrF + ShldrS + ElbowR + Elbow
                angles      = move_arms(robot, Arms, angles, minmaxArms)
            if move_loop.get(108):# L - Legs
                angles      = [
                    angle_R_HipF,   angle_L_HipF,
                    angle_R_HipS,   angle_L_HipS,
                    angle_R_HipR,   angle_L_HipR,
                    angle_R_Knee,   angle_L_Knee,
                    angle_R_AnkleF, angle_L_AnkleF,
                    angle_R_AnkleS, angle_L_AnkleS,]
                minmaxLegs = [__HipF, [R_HipS, L_HipS], __HipR, __Knee, __AnkleF, __AnkleS]
                Legs       = HipF + HipS + HipR + Knee + AnkleF + AnkleS
                angles     = move_legs(robot, Legs, angles, minmaxLegs)
            if move_loop.get(49): # Quit MV Block
                move_flag = False
            if move_loop.get(47):
                msg = 'MV Block: press "1" to go "Main"'
                printer(msg)
            time.sleep(0.3)
    if keys.get(47):
        printer(msg)
    if keys.get(113):
        printer('Quit Main...')
        main_flag = False
    time.sleep(0.1)
p.disconnect()